In [12]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

In [9]:
data_df = pd.read_csv('rearranged_optimal_data_set.csv')

In [10]:
data=np.array(data_df)
n = data.shape[0]
d = data.shape[1]
d -= 1
n_train = int(n*0.7)
n_test = int(n*0.2)
n_validate  = n - n_train - n_test
np.random.seed(101)
data = np.random.permutation(data)

X_train = np.zeros((n_train,d))
X_test  = np.zeros((n_test,d))
X_validate = np.zeros((n_validate,d))
Y_train = np.zeros((n_train))
Y_test = np.zeros((n_test))
Y_validate = np.zeros((n_validate))

X_train[:] = data[:n_train,:-1]
Y_train[:] = data[:n_train,-1]

X_test[:] = data[n_train:(n_train+n_test),:-1]
Y_test[:] = data[n_train:(n_train+n_test),-1]

X_validate[:] = data[(n_train+n_test):, :-1]
Y_validate[:] = data[(n_train+n_test):,-1]

print(X_train.shape)
print(X_test.shape)
print(X_validate.shape)
print(Y_train.shape)
print(Y_test.shape)
print(Y_validate.shape)

(28831, 81)
(8237, 81)
(4120, 81)
(28831,)
(8237,)
(4120,)


In [23]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train)

In [24]:
print(classification_report(y_true=Y_test, y_pred=clf.predict(X_test)
                            , target_names=['no', 'yes']))

             precision    recall  f1-score   support

         no       0.94      0.94      0.94      7279
        yes       0.54      0.53      0.53       958

avg / total       0.89      0.89      0.89      8237



In [50]:
clf1 = tree.DecisionTreeClassifier(min_samples_leaf = 100)
clf1 = clf1.fit(X_train, Y_train)

In [51]:
print(classification_report(y_true=Y_test, y_pred=clf1.predict(X_test)
                            , target_names=['no', 'yes']))

             precision    recall  f1-score   support

         no       0.93      0.97      0.95      7279
        yes       0.66      0.45      0.54       958

avg / total       0.90      0.91      0.90      8237



In [52]:
data_df1 = pd.read_csv('rearranged_whole_data_set.csv')

In [53]:
data=np.array(data_df1)
n = data.shape[0]
d = data.shape[1]
d -= 1
n_train = int(n*0.7)
n_test = int(n*0.2)
n_validate  = n - n_train - n_test
np.random.seed(101)
data = np.random.permutation(data)

X_train = np.zeros((n_train,d))
X_test  = np.zeros((n_test,d))
X_validate = np.zeros((n_validate,d))
Y_train = np.zeros((n_train))
Y_test = np.zeros((n_test))
Y_validate = np.zeros((n_validate))

X_train[:] = data[:n_train,:-1]
Y_train[:] = data[:n_train,-1]

X_test[:] = data[n_train:(n_train+n_test),:-1]
Y_test[:] = data[n_train:(n_train+n_test),-1]

X_validate[:] = data[(n_train+n_test):, :-1]
Y_validate[:] = data[(n_train+n_test):,-1]

print(X_train.shape)
print(X_test.shape)
print(X_validate.shape)
print(Y_train.shape)
print(Y_test.shape)
print(Y_validate.shape)

(28831, 102)
(8237, 102)
(4120, 102)
(28831,)
(8237,)
(4120,)


In [58]:
clf2 = tree.DecisionTreeClassifier(min_samples_leaf = 100)
clf2 = clf2.fit(X_train, Y_train)

In [59]:
print(classification_report(y_true=Y_test, y_pred=clf2.predict(X_test)
                            , target_names=['no', 'yes']))

             precision    recall  f1-score   support

         no       0.93      0.97      0.95      7279
        yes       0.66      0.45      0.54       958

avg / total       0.90      0.91      0.90      8237

